<a href="https://colab.research.google.com/github/minami1009/bio/blob/main/upgma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#preparation of GoogleColab
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
file_path="/content/drive/MyDrive/USalign_result3.csv"
df=pd.read_csv(file_path, sep='\t')
print(df)

In [ ]:
#df["TM_diff"]=abs(df["TM1"]-df["TM2"])
#df["TM*ID_diff"]=abs(df["TM1"]*df["L1"]-df["TM2"]*df["L2"])
df["TM_ave"]=(df["TM1"]*df["L1"]+df["TM2"]*df["L2"])/(df["L1"]+df["L2"])
#print(df.describe())

plt.style.use('default')
sns.set()
sns.set_style('whitegrid')
sns.set_palette('gray')

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.violinplot(df["TM_ave"])
plt.show()


In [ ]:
max_value=df["TM_ave"].max()
min_value=df["TM_ave"].min()
df["TM_ave_scaled"]=1-(df["TM_ave"]-min_value)/(max_value-min_value)

plt.style.use('default')
sns.set()
sns.set_style('whitegrid')
sns.set_palette('gray')

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.violinplot(df["TM_ave_scaled"])
plt.show()

In [ ]:
distances = df.pivot(index='PDBchain1', columns='PDBchain2', values='TM_ave_scaled')

# 距離行列を作成
distance_matrix = distances.values

# 距離行列にラベルを追加
dm_df = pd.DataFrame(distance_matrix, index=distances.index, columns=distances.columns)

print(dm_df)

In [ ]:
#1列目と最終行に追加
dm_df.insert(0,"A0A370CPH5.pdb:A",0)  #55T2の場合はB8XY56.pdb:A
dm_df.loc["X8FA76.pdb:A"]=0 #55T2の場合はQ9NR97.pdb:A

print(dm_df)

In [ ]:
#対角の成分は0にし、i,jをj,iの値で置き換える
for i in list(range(len(dm_df))):
  dm_df.iloc[i,i] = 0
  for j in list(range(i)):
    dm_df.iat[i,j]=dm_df.iat[j,i]
print(dm_df)

In [ ]:
namefile_path="/content/drive/MyDrive/name.csv"
df_name=pd.read_csv(namefile_path)
print(df_name)

In [ ]:
entry_list = list(dm_df.columns.values)
print(entry_list)
def id_to_name(entry_list, df_name):
  # 変換したOrganismのリストを格納する空のリストを作成
  transformed_organism_list = []

  # Entryのリストに対応するOrganismを探し、変換したリストを作成
  for entry in entry_list:
    try:
      transformed_organism = df_name.loc[df_name['Entry'] == entry[:-6], 'Organism'].values[0]
      transformed_organism_list.append(transformed_organism)
    except:
      transformed_organism = entry
      transformed_organism_list.append(transformed_organism)

  # 結果のリストを表示
  return transformed_organism_list

organism_list = id_to_name(entry_list, df_name)
print(organism_list)

In [ ]:
import numpy as np
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from google.colab import files


# 距離行列を作成（ここでは仮のデータを使用）
distance_matrix = dm_df

# 階層的クラスタリングを実行
# linkage関数では階層的クラスタリングアルゴリズムを指定します（ここではward法を使用）
Z = linkage(distance_matrix, method='average')
plt.figure(figsize=[10,30])

# 樹形図を作成
#dendrogram(Z,orientation="left",labels=list(dm_df.columns.values))
dendrogram(Z,orientation="left",labels=organism_list)

# グラフを表示
plt.title('Dendrogram')
plt.xlabel('Data Points')
plt.ylabel('Distance')
plt.rcParams["font.size"] = 36
#plt.show()
#plt.axis('off')

#plt.savefig("tree.png")
#files.download("tree.png")